**Part 1: Analyze the return of a portfolio**

At the end of 2004 a retail investor (that has EUR as a reference currency) bought a portfolio of 100 stocks for 1% each, as indicated in the "Stocks" sheet in the Excel file. He kept the stocks until the end of February 2020, but at the end of each month he rebalanced the portfolio such that each stock’s weight went back to its original 1% weight.

The portfolio return is judged against a “benchmark” that has starting weights (as of end
December 2004) as shown in the "Stocks" sheet. The benchmark is not rebalanced (i.e. the weights “drift” over time with the performance of the different stocks). When a stock gets delisted during a month, at the end of that same month the stock is sold (at the price at that time) and the proceeds distributed over all the other stocks, proportionally with the weight they have at that time.

Your report should document the performance of the portfolio, including at least the following:

● The total return of the portfolio and the total return of the benchmark over the entire period; the annualized return for both. Make sure you calculate the returns in EUR. **(Worksheet F14 for returns)**

● The decomposition of the excess return (compared to the benchmark) into the contributions from the over- and underweighting of the different individual stocks. **(Difference between Portfolio and Benchmark retruns)**

● The sectors and regions that contributed most to the return difference. **(Worksheet Stocks)**

● The drivers of the excess return: over/underweighting the sector/region compared to the benchmark, or better/worse individual stock selection? 

● Max drawdown, best/worst months, annualized volatility, etc.

You can add additional information you deem relevant for the analysis of the portfolio and use figures and tables when desirable.


**Data**

**The Excel workbook available here contains the following sheets:**

● **Stocks**: a list of 1539 stocks, including some basic stock information; sector, region, currency and when the stocks were delisted (if applicable).

● **Factors**: A list of 13 fundamental stock characteristics (“factors”) and 2 types of returns (in EUR and in local currency) over time.

● **Other sheets**: contain end-of-month data for the stocks and factors between December 2004 and February 2020.

In [10]:
import pandas as pd
import numpy as np

tab = 'F14'

def read_returns(sheet_name = 'datenaufgabe.xlsx', tab = 'F14'):
    """"[Reads contents from 'datenaufgabe.xlsx']

    Returns:
        [Returns sheet 'F14' from 'datenaufgabe.xlsx' as DataFrame]
    
    pass"""
    
    return pd.read_excel(sheet_name, tab)


# Here you should get a DataFrame with datetime index    
data = read_returns(tab = tab)
print(data)

           F14  IS0001  IS0002  IS0003  IS0004  IS0005  IS0006  IS0007  \
0   2004-12-31   -4.34   -1.76    0.20    2.48   10.43    1.03    0.46   
1   2005-01-30   11.21   14.72   -5.62    0.92   -3.19    3.94   -8.69   
2   2005-02-28    6.29    5.69   -0.67    6.36    3.49   -3.58   -2.43   
3   2005-03-31  -14.20   -1.53   -2.90    4.27   -9.14   -2.37    8.55   
4   2005-04-30   -8.77   -2.12   10.50    3.80    0.80   -0.06   -1.75   
..         ...     ...     ...     ...     ...     ...     ...     ...   
178 2019-10-31   -7.32  -12.86   -3.28    0.00    0.89    0.00   -4.20   
179 2019-11-30   12.11    0.27    7.48    0.00   -1.13    0.00    4.45   
180 2019-12-31  -12.74   -7.11   -3.25    0.00   -3.61    0.00  -15.10   
181 2020-01-31    5.78    0.36    0.99    0.00    3.60    0.00    8.48   
182 2020-02-28    2.13    4.65    7.18    0.00    0.49    0.00    5.91   

     IS0008  IS0009  ...  IS1530  IS1531  IS1532  IS1533  IS1534  IS1535  \
0      2.49    2.57  ...    5.28   

For reading excel with pandas look at [pandas.read_excel](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html). In this case i would recommend reading into a [time series](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html).

You probably will need to do sth like `dates = returns.pop('F14')` where I assume `F14` is the column name that holds the dates. Then you can turn it to a DatetimeIndex like
```python
idx = pd.DatetimeIndex(dates.values)
```
and then set new index with

```python
returns.set_index(idx)
```
`read_excel` probably provides a more elegant solution but the above should work



For the calculation of total return you should have a look at [pandas.DataFrame.diff](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.diff.html)

In [51]:
# Here i would turn it into a time series (as explained above) for convenience like this:

dates = data.pop('F14')
idx = pd.DatetimeIndex(dates.values)
timeseries = data.set_index(idx)

# data as time series
timeseries

KeyError: 'F14'

In [46]:
# With time series it's easy to access time frames, you might need this for annual return
# Like e.g:
timeseries['2019']

,IS0001,IS0002,IS0003,IS0004,IS0005,IS0006,IS0007,IS0008,IS0009,IS0010,...,IS1530,IS1531,IS1532,IS1533,IS1534,IS1535,IS1536,IS1537,IS1538,IS1539
2019-01-31,-1.26,0.38,5.48,5.23,-1.57,0.0,1.18,7.20,6.63,3.00,...,6.76,10.77,-0.09,-30.32,1.54,2.45,0.40,4.20,3.31,2.58
2019-02-28,-6.35,-10.72,-0.87,0.41,-2.20,0.0,2.34,-4.39,3.60,-7.17,...,-1.74,-3.11,-1.28,-0.98,-6.59,4.30,-0.24,2.47,-0.16,-3.42
2019-03-30,-6.89,-3.57,-1.50,-5.38,-1.43,0.0,-5.45,-11.50,-7.68,-3.93,...,1.14,-8.05,3.74,-3.68,-6.77,-4.87,-0.56,-1.68,-7.69,-7.56
2019-04-30,8.32,4.45,-0.78,0.22,2.37,0.0,1.16,12.48,6.89,0.97,...,8.65,-26.97,4.15,-1.58,7.51,5.69,0.00,5.81,9.55,-9.86
2019-05-31,-4.21,-0.04,9.56,0.55,2.29,0.0,6.61,-6.89,2.13,-0.30,...,-14.11,12.36,-2.94,-11.34,0.21,4.05,0.00,-4.11,1.20,5.73
2019-06-29,-16.66,-3.59,-0.90,0.03,-1.41,0.0,5.02,13.52,-10.23,-2.26,...,-2.77,-4.35,-3.84,5.98,0.13,-3.89,0.00,-0.37,-4.15,-0.28
2019-07-31,15.28,4.70,7.71,-0.03,0.81,0.0,-2.81,3.67,4.90,9.76,...,-0.81,-4.97,1.15,1.84,12.39,-2.09,0.00,3.70,4.32,7.70
2019-08-31,8.12,3.58,2.56,0.00,0.42,0.0,6.72,3.16,-2.22,-2.66,...,16.80,3.64,10.68,7.02,-0.95,4.25,0.00,-0.27,-5.74,0.58
2019-09-28,-9.02,0.66,9.95,0.00,1.05,0.0,0.84,2.93,2.60,-0.50,...,3.75,2.39,4.81,-8.37,6.72,-5.73,0.00,4.22,5.53,0.07
2019-10-31,-7.32,-12.86,-3.28,0.00,0.89,0.0,-4.20,-4.46,-8.59,-4.19,...,-4.64,0.91,1.94,-29.11,-11.43,-3.83,0.00,0.50,-6.28,-7.44


In [47]:
# Or e.g like this, pandas does a lot of work under the hub
timeseries['2019-08-01':'2019-12-31']

,IS0001,IS0002,IS0003,IS0004,IS0005,IS0006,IS0007,IS0008,IS0009,IS0010,...,IS1530,IS1531,IS1532,IS1533,IS1534,IS1535,IS1536,IS1537,IS1538,IS1539
2019-08-31,8.12,3.58,2.56,0.0,0.42,0.0,6.72,3.16,-2.22,-2.66,...,16.80,3.64,10.68,7.02,-0.95,4.25,0.0,-0.27,-5.74,0.58
2019-09-28,-9.02,0.66,9.95,0.0,1.05,0.0,0.84,2.93,2.60,-0.50,...,3.75,2.39,4.81,-8.37,6.72,-5.73,0.0,4.22,5.53,0.07
2019-10-31,-7.32,-12.86,-3.28,0.0,0.89,0.0,-4.20,-4.46,-8.59,-4.19,...,-4.64,0.91,1.94,-29.11,-11.43,-3.83,0.0,0.50,-6.28,-7.44
2019-11-30,12.11,0.27,7.48,0.0,-1.13,0.0,4.45,8.74,-3.02,7.14,...,-6.59,6.32,2.49,-0.28,3.09,4.12,0.0,0.81,-5.15,10.10
2019-12-31,-12.74,-7.11,-3.25,0.0,-3.61,0.0,-15.10,-8.88,-5.33,-3.76,...,-6.58,9.02,-1.27,1.30,-12.00,-17.07,0.0,-6.11,-6.51,-9.23


In [14]:
# or for a Stock Code
timeseries['IS0001']

2004-12-31    -4.34
2005-01-30    11.21
2005-02-28     6.29
2005-03-31   -14.20
2005-04-30    -8.77
              ...  
2019-10-31    -7.32
2019-11-30    12.11
2019-12-31   -12.74
2020-01-31     5.78
2020-02-28     2.13
Name: IS0001, Length: 183, dtype: float64

In [15]:
# or a Stock Code and a time period
timeseries['IS0001']['2019-08-01':'2019-12-31']

2019-08-31     8.12
2019-09-28    -9.02
2019-10-31    -7.32
2019-11-30    12.11
2019-12-31   -12.74
Name: IS0001, dtype: float64

In [16]:
timeseries['IS0001']

2004-12-31    -4.34
2005-01-30    11.21
2005-02-28     6.29
2005-03-31   -14.20
2005-04-30    -8.77
              ...  
2019-10-31    -7.32
2019-11-30    12.11
2019-12-31   -12.74
2020-01-31     5.78
2020-02-28     2.13
Name: IS0001, Length: 183, dtype: float64

In [18]:
# I added timeseries as parameter

def total_return(timeseries, portfolio):
    """[Returns the total return of the portfolio over the entire period]
    
    Arguments:
        portfolio {[list]} -- [List of Stock Codes for portfolio]
    
    Returns:
        [Returns total return over entire period]
    """
    
    # a variable to hold your return value
    total = 0
    
    # iterate over your portfolio's stock codes
    for stock_code in portfolio:
        
        # for a better understanding of the control flow you can always add print statements
        print(stock_code)
        
        # e.g. print the values for the stock code in timeseries
        print(timeseries[stock_code])
        
        # which you can also assign to a variable
        stock_values = timeseries[stock_code]
        
        # here you need to calculate the sum of all elements in 'stock_values'
        sum_of_all_values = 0 # TODO Change this!!! Look up pandas do
        
        # and sum it up to your total like
        total = total + sum_of_all_values
    
    # finally you need to return total
    return total
        

# Your portfolio parameter is just a list of Stock Codes
portfolio = ['IS0001']

# So you can call your function like this

total = total_return(timeseries, portfolio)

IS0001
2004-12-31    -4.34
2005-01-30    11.21
2005-02-28     6.29
2005-03-31   -14.20
2005-04-30    -8.77
              ...  
2019-10-31    -7.32
2019-11-30    12.11
2019-12-31   -12.74
2020-01-31     5.78
2020-02-28     2.13
Name: IS0001, Length: 183, dtype: float64


In [34]:
log_returns = (timeseries['IS0001']/timeseries['IS0001'].shift(1))
log_returns.dropna(inplace=True)
log_returns

2005-01-30   -2.582949
2005-02-28    0.561106
2005-03-31   -2.257552
2005-04-30    0.617606
2005-05-31   -0.151653
                ...   
2019-10-31    0.811530
2019-11-30   -1.654372
2019-12-31   -1.052023
2020-01-31   -0.453689
2020-02-28    0.368512
Name: IS0001, Length: 182, dtype: float64

In [42]:
# get returns of the stock code in timeseries
log_returns = (timeseries['stock_code']/timeseries['stock_code'].shift(1))
log_returns.dropna(inplace=True)
    
    
    #to get mean daily returns
mean_daily_returns = log_returns.mean()
mean_daily_returns    

2.589356956639417

In [50]:
prets = np.sum(mean_daily_returns * 100 * 252) #mal 100 da 100 stocks
prets

65251.79530731331

In [40]:
# I added timeseries as parameter

def total_return(timeseries, portfolio):
    """[Returns the total return of the portfolio over the entire period]
    
    Arguments:
        portfolio {[list]} -- [List of Stock Codes for portfolio]
    
    Returns:
        [Returns total return over entire period]
    """
    
    # get returns of the stock code in timeseries
    log_returns = (timeseries['IS0001']/timeseries['IS0001'].shift(1))
    log_returns.dropna(inplace=True)
    
    
    #to get mean daily returns
    mean_daily_returns = log_returns.mean()
    
    portfolio = ['IS0001']
    
    #a variable to hold your return value
    results = np.zeros((99,portfolio))  
      
    # iterate over your portfolio's stock codes
    for i in enumerate(portfolio):
               
        #get portfolio total return, multiply with 100 since 
        prets = np.sum(mean_daily_returns * sum_of_all_values * 252)
        
        # here you need to calculate the sum of all elements in 'stock_values'
        sum_of_all_values = 100 # TODO Change this!!! Look up pandas do
        
        # and store results in array
        total[0,i] = prets
    
    # finally you need to return total
    return total
        
# So you can call your function like this
total = total_return(timeseries, portfolio)

TypeError: 'list' object cannot be interpreted as an integer

To keep it simple, I would implement the annual return as `total_return` and also pass the year as parameter. If you managed to get `returns` as time series you can now do:

```python
returns_2019 = returns['2019-01-01':'2019-12-31']
```

In [3]:
def annual_return(portfolio, year):
    """[Returns the annual return of the portfolio over the given year]
    
    Arguments:
        portfolio {[list]} -- [List of Stock Codes for portfolio]
        year {[string]} -- [String representing the year]
    
    Returns:
        [Returns annual return for given year]
    """
    pass